In [ ]:
import random
import numpy as np

class NN:
    def __init__(self, activation_fun, leaky_alpha=0.01):
        """
        Initializes a neural network object.
        Parameters:
        - activation_fun (str): The activation function to be used in the network.
                               Options: "relu", "leaky_relu", or "sigmoid".
        - leaky_alpha (float): The slope for the leaky ReLU activation function (default is 0.01).
        """
        self.num_layers = 1 # input layer 
        self.neurons_per_layer = []
        self.biases = []
        self.weights = []
        self.activation_fun_name = activation_fun  # store activation function name

        # Set the activation function and its derivative
        if self.activation_fun_name == "relu":
            self.activation_fun = lambda x: np.maximum(x, 0)
            self.activation_fun_d = lambda x: np.where(x > 0, 1, 0)
        elif self.activation_fun_name == "leaky_relu":
            self.leaky_alpha = leaky_alpha
            self.activation_fun = lambda x: np.where(x > 0, x, self.leaky_alpha * x)
            self.activation_fun_d = lambda x: np.where(x > 0, 1, self.leaky_alpha)
        elif self.activation_fun_name == "softplus":
            self.activation_fun = lambda x: np.log(1 + np.exp(x))
            self.activation_fun_d = lambda x: 1 / (1 + np.exp(-x))
        elif self.activation_fun_name == "sigmoid":
            self.activation_fun = lambda x: 1 / (1 + np.exp(-x))
            self.activation_fun_d = lambda x: self.activation_fun(x) * (1 - self.activation_fun(x))
            
    
    def layer(self, num_inputs, num_neurons):
        """Adds a new layer to the neural network.
        Parameters:
        num_inputs (int): Number of inputs or number of neurons of previous layer.
        num_neurons (int): Number of neurons in the layer."""
        if self.num_layers == 1:
            self.neurons_per_layer.append(num_inputs)
        self.num_layers += 1
        self.neurons_per_layer.append(num_neurons)
        self.biases.append(np.random.randn(num_neurons, 1)*0.1) # gaussian distribution with mean 0 and standard deviation 0.1
        self.weights.append(np.random.randn(num_neurons, num_inputs)*0.1)
        
    def forwardpass(self, x):
        """
        Performs a forward pass through the neural network.
        Parameters:
        - x (ndarray): Input data.
        Returns:
        - ndarray: Output of the neural network.
        """
        if self.activation_fun_name in ["relu", "leaky_relu", "softplus", "sigmoid"]:
            for i in range(len(self.biases)-1):
                x = self.activation_fun(np.dot(self.weights[i], x) + self.biases[i])
            x = np.dot(self.weights[-1], x) + self.biases[-1]
        #code for output layer with activation function
        else:
            for i in range(len(self.biases)):
                x = self.activation_fun(np.dot(self.weights[i], x) + self.biases[i])
        return x
    
    # loss function - MSE
    def loss(self, x, y):
        return (self.forwardpass(x) - y)**2
    
    def mean_loss(self, X, Y):
        return np.mean([self.loss(x, y) for x, y in zip(X, Y)])
    
    def loss_d(self, activation, y):
        return (activation - y) 
    
    def backprop(self, x, y):
        """
        Performs backpropagation through the neural network.
        Parameters:
        - x (ndarray): Input data.
        - y (ndarray): Target data.
        """
        grad_b = [np.zeros(b.shape) for b in self.biases] # initialize list for partial derivatives of the cost function with respect to the biases
        grad_w = [np.zeros(w.shape) for w in self.weights] # initialize list for partial derivatives of the cost function with respect to the weights
        nets = []
        activations = [x] #first layer - x is input layer 
        if self.activation_fun_name in ["relu", "leaky_relu", "softplus", "sigmoid"]:
            # forward pass
            for i in range(len(self.biases)-1):
                net = np.dot(self.weights[i], activations[-1]) + self.biases[i]
                nets.append(net)
                activations.append(self.activation_fun(net))
            # output layer without activation function
            net = np.dot(self.weights[-1], activations[-1]) + self.biases[-1]
            nets.append(net)
            activations.append(net)
            #backward pass - only for output layer
            error = self.loss_d(activations[-1], y)
            grad_b[-1] = error
            grad_w[-1] = np.dot(error, activations[-2].T)
        # code for output layer with activation function
        else:
            for i in range(len(self.biases)):
                net = np.dot(self.weights[i], activations[-1]) + self.biases[i]
                nets.append(net)
                activations.append(self.activation_fun(net))
            error = self.loss_d(activations[-1], y) * self.activation_fun_d(nets[-1])
            grad_b[-1] = error
            grad_w[-1] = np.dot(error, activations[-2].T)
        # backward pass for rest of the layers
        for i in range(2, self.num_layers-1):
            error = np.dot(self.weights[-i+1].T, error) * self.activation_fun_d(nets[-i])
            grad_b[-i] = error
            grad_w[-i] = np.dot(error, activations[-i-1].T)
        return grad_b, grad_w

    def fit(self, X, Y, epochs, learning_rate):
        """
        Trains the neural network using full batch gradient descend.
        Parameters:
        - X (ndarray): Input data.
        - y (ndarray): Target data.
        - epochs (int): Number of epochs.
        - learning_rate (float): Learning rate.
        """
        for epoch in range(epochs):
            grad_b_sum = [np.zeros(b.shape) for b in self.biases]
            grad_w_sum = [np.zeros(w.shape) for w in self.weights]
            for x,y in zip(X,Y):
                grad_b, grad_w = self.backprop(x, y)
                grad_b_sum = [gb + g for gb, g in zip(grad_b_sum, grad_b)]
                grad_w_sum = [gw + g for gw, g in zip(grad_w_sum, grad_w)]
            self.biases = [b - (learning_rate/len(x)) * gb for b, gb in zip(self.biases, grad_b_sum)]
            self.weights = [w - (learning_rate/len(x)) * gw for w, gw in zip(self.weights, grad_w_sum)]
            if (epoch+1) % 10 == 0:
                print("Epoch: {}, Loss: {}".format(epoch+1, 
                                                   self.mean_loss(X, Y)))
    def test(self, X, Y):
        """
        Tests the neural network.
        Parameters:
        - X (ndarray): Input data.
        - y (ndarray): Target data.
        """
        for x,y in zip(X,Y):
            print("Target: {}, Prediction: {}".format(y, self.forwardpass(x)))

In [ ]:
#import data
import pandas as pd
data = pd.read_csv('CrabAgePrediction.csv')

In [ ]:
# data preprocessing
data_target = data['Age']
data = data.drop(columns=['Age'])
data['Sex'] = data['Sex'].map({'F': -1, 'I': 0, 'M': 1})

In [ ]:
#normalize data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.loc[:, data.columns != 'Sex'])
data_scaled = pd.DataFrame(data_scaled)
data_scaled = pd.concat([data_scaled, data['Sex']], axis=1)

In [ ]:
#change data format for NN input
x = [np.reshape(row, (8, 1)) for row in data_scaled.values]
y = [np.reshape(row, (1, 1)) for row in data_target.values]

In [ ]:
#split data into train and test set
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#create NN models with different layers and neurons
models = []
i = 0 
for layers in reversed(range(1,5)):
    for neurons in reversed([4,8,16,32]):
        for activation in ["relu"]:
            for iter in range(4):
                i += 1 
                model = NN(activation)
                model.layer(8, neurons)
                for _ in range(layers-1):
                    model.layer(neurons, neurons)
                model.layer(neurons, 1)
                model.fit(x_train, y_train, 500, 0.0001)
                print("{}/64\nLayers: {}, Neurons: {}, Activation: {}, Loss: {}, Iter = {}".format(i,layers, neurons, activation, model.mean_loss(x_test, y_test), iter+1))
                models.append(model)

In [ ]:
#create NN models with different activation functions
models_activations = []
i = 0 
for layers in [2]:
    for neurons in [32]:
        for activation in ["relu", "leaky_relu", "softplus", "sigmoid"]:
            for iter in range(4):
                i += 1 
                model = NN(activation)
                model.layer(8, neurons)
                for _ in range(layers-1):
                    model.layer(neurons, neurons)
                model.layer(neurons, 1)
                model.fit(x_train, y_train, 1000, 0.0001)
                print("{}/48\nLayers: {}, Neurons: {}, Activation: {}, Loss: {}, Iter = {}".format(i,layers, neurons, activation, model.mean_loss(x_test, y_test), iter+1))
                models_activations.append(model)

In [ ]:
#save results to excel file
df = pd.read_excel("Wyniki.xlsx")
for model in models:    
    df_result = pd.DataFrame([model.mean_loss(x_test, y_test), model.num_layers, model.neurons_per_layer, model.activation_fun_name]).T
    df_result.columns = ['Loss', 'Layers', 'Neurons', 'Activation']
    df = pd.concat([df, df_result], ignore_index=True)
df.to_excel("Wyniki.xlsx", index=False)

In [ ]:
#save results to excel file
df = pd.read_excel("Activations.xlsx")
for model in models_activations:    
    df_wynik = pd.DataFrame([model.mean_loss(x_test, y_test), model.num_layers, model.neurons_per_layer, model.activation_fun_name]).T
    df_wynik.columns = ['Loss', 'Layers', 'Neurons', 'Activation']
    df = pd.concat([df, df_result], ignore_index=True)
df.to_excel("Activations.xlsx", index=False)